This has code from online examples with landlab
see other rain_sample_topo file for a more integrated rain and overland flow model

In [ ]:
# PYTHON IMPORTS
import numpy as np #basic python library
import matplotlib.pyplot as plt #For plotting results
from matplotlib.pyplot import figure, legend, plot, show, title, xlabel, ylabel, ylim
import cv2 
from random import randint

# LANDLAB COMPONENTS 
from landlab.components import DepressionFinderAndRouter #Pit filling
from landlab.components import PriorityFloodFlowRouter #Flow routing
from landlab.components import FlowAccumulator #FlowAccumulator 
from landlab.components import ChannelProfiler 
from landlab.components import SteepnessFinder
from landlab.components import ChiFinder
from landlab.components import ExponentialWeatherer
from landlab.components import DepthDependentDiffuser
from landlab.components import SpatialPrecipitationDistribution
from landlab.components import OverlandFlow
from landlab.utils.flow__distance import calculate_flow__distance

# SPACE COMPONENTS 
from landlab.components import Space #SPACE model
from landlab.components import SpaceLargeScaleEroder #basically SPACE 2.0 -- use this 
from landlab.components import FastscapeEroder #calculates the amount of erosion at each node
from landlab.components import SinkFillerBarnes #replaces pits with shallow gradients for draining

# LANDLAB UTILITIES 
from landlab import RasterModelGrid #Grid utility
from landlab import imshow_grid #For plotting results
from landlab.io import read_esri_ascii #to read in an ascii file
from landlab.io import read_asc_header #to read in the header of the ascii file
from landlab.io import write_esri_ascii 

%matplotlib inline

In [ ]:
# GRID SPECS
ncols = 100
nrows = 100
cellsize = 10

# RETREIEV TOPO FROM ASCII
#path_project = r"C:/Users/gsbir/Documents/EES_393/Export/" # add the path to the asc file    #this should access my H drive
topo_dem = "sample_topo2_topographic__elevation.txt" #the name of the ascii file
topo_path = topo_dem
topo_header = read_asc_header(open(topo_path, 'r'))
(mg, z) = read_esri_ascii(topo_path, name = 'topographic__elevation', halo = 0) #generating the new grid

# FIND OUTLET
open_nodes = mg.core_nodes
min_topo = mg['node']['topographic__elevation'].argmin()
min_topo_ID = open_nodes[min_topo]
outlet_id = mg.set_watershed_boundary_condition_outlet_id(min_topo_ID, mg['node']['topographic__elevation'], -9999)
outlet_id = min_topo_ID

# HYDROLOGICALLY FILL THE DEM
sfb = SinkFillerBarnes(mg, surface=z, method='D8', fill_flat=False) #creating an instance of sinkfillerbarnes 
sfb.run_one_step() #This is important to ensure that your future flow routing will work properly. - run the sinkfiller barnes once

# CLOSE BOUNDARIES
mg.set_closed_boundaries_at_grid_edges(bottom_is_closed=True,left_is_closed=True,right_is_closed=True,top_is_closed=True)

# PLOT THE TOPOGRAPHY 
figelev = plt.figure()#Instantiate figure as empty plot
plot = plt.subplot()#Instantiate subplot as empty plot
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)

In [ ]:
# PARAMETERS FOR SPACE
K_sed = 0.0001
K_sp = 0.001
K_br = 0.0001
F_f = 0.0
phi = 0.0
H_star = 1.0
v_s = 5.0
m_sp = 0.5
n_sp = 1.0
sp_crit_sed = 0
sp_crit_br = 0

#INSTANTIATING COMPONENTS 
fr = PriorityFloodFlowRouter(mg, flow_metric='D8', suppress_out = True)

fa = FlowAccumulator(mg, flow_director='D8') 

df = DepressionFinderAndRouter(mg,pits = 'flow__sink_flag', reroute_flow = True) 

fsc = FastscapeEroder(mg, K_sp, m_sp, n_sp)

#RUNNING FSC FOR 1 YEAR TO IDENTIFY CHANNELS 
fa.run_one_step()
df.map_depressions()
fsc.run_one_step(1)

# SET UP CHANNEL PROFILER AND RUN 
profiler = ChannelProfiler(mg,number_of_watersheds=1,minimum_channel_threshold=0,main_channel_only=True)
profiler.run_one_step()
profiler.data_structure.keys() #is [101]
profiler.data_structure[101].keys()
profiler.data_structure[101][(101,9687)]["ids"] #produces array of id values for the channel


In [ ]:
#ADDING REST OF THE FIELDS
#Add field 'soil__depth' to the grid
mg.add_zeros('node', 'soil__depth')
mg.at_node['soil__depth'][:] = 2  #Set 2 m of initial soil depth at core nodes

#Add field 'bedrock__elevation' to the grid
mg.add_zeros('bedrock__elevation', at='node')
mg.at_node["bedrock__elevation"][:] = mg.at_node["topographic__elevation"] - mg.at_node['soil__depth']
#mg.at_node["bedrock__elevation"][:] -= mg.at_node['soil__depth']

mg.at_node['bedrock__elevation'][outlet_id] = 0
mg.at_node['topographic__elevation'][outlet_id] = 0
mg.at_node['soil__depth'][outlet_id] = 0
#Add field 'bedrock__elevation' to the grid
mg.add_zeros('surface_water__depth', at='node')
print(outlet_id)


Setting Up Rain and Overland Flow Parameters

In [ ]:
of = OverlandFlow(mg, steep_slopes=True)
rain = SpatialPrecipitationDistribution(mg, number_of_years=1)
#storm_t = 1.5 #durration of a storm in hr
#interstorm_t = 8 #time between storms in hr
rain.total_time = 1 #this is for the rain.yield_storms (hr)
of_total_time = 3600 * 8#my own variable for how long overland flow runs in (s) -> 3600 s in 1 h



use precipitation component

the following code created a series of random storm events ( diff amt of storms each iteration )
- each plot produced is one storms' rainfall

In [ ]:

np.random.seed(randint(0, 100)) 

# get the storm simulator to provide a storm

storm_time_data = []
interstorm_time_data = []
storm_amt_data = []
x = 1 #to keep track of each iteration of loop


#this is running for a year
for (storm_t, interstorm_t) in rain.yield_storms(style="whole_year"):  # storm lengths in hrs
    
    storm_time_data.append(storm_t)
    interstorm_time_data.append(interstorm_t*.041)
    storm_amt_data.append(x)
    mg.at_node['rainfall__flux'] *= 0.001  # because the rainfall comes out in mm/h
    mg.at_node['rainfall__flux'] *= 2.0  # to make the storm heavier
    print(rain.median_total_rainfall_this_year)
    print(rain.total_time)
    plt.figure()# plot up this storm
    imshow_grid(mg, 'rainfall__flux', cmap='gist_ncar', colorbar_label='Rainfall flux (mm/h)')
    plt.show()
    x+=1

PLOT OVERALL PRECIPITATION TIME VS. STORM LENGTH

each dot on the graph corresponds to a rainflux plot above

In [ ]:
#show graphs of storm time and interstorm data
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()
plot.plot(storm_amt_data, storm_time_data,linestyle='--', marker='o')
plt.xlabel("Storm Event")
plt.ylabel("Duration of Storm (hr)")
plt.xticks(np.arange(1,x,1))
print(storm_time_data)
total_t_check = sum(storm_time_data)

PLOT OVERALL PRECIPITATION TIME VS. INTERSTORM TIME

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()
plot.plot(storm_amt_data, interstorm_time_data,linestyle='--', marker='o')
plt.xlabel("Storm Event")
plt.ylabel("Interstorm Duration (hr)")
plt.xticks(np.arange(1,x,1))
print(interstorm_time_data)
total_t_check += sum(interstorm_time_data)
print(total_t_check)

In [ ]:
outlet_node_to_sample = 304
outlet_link_to_sample = mg.links_at_node[outlet_node_to_sample][3]
upstream_node_to_sample = 1954
upstream_link_to_sample = mg.links_at_node[upstream_node_to_sample][3]
midstream_node_to_sample = 6596
midstream_link_to_sample = mg.links_at_node[midstream_node_to_sample][3]



## Lists for saving data
discharge_at_outlet = []
discharge_upstream = []
discharge_midstream = []
hydrograph_time = []

## Setting initial fields...
mg["node"]["surface_water__discharge"] = np.zeros(mg.number_of_nodes)

In [ ]:
plt.figure(1)
imshow_grid(mg, z)  # plot the DEM
plt.plot(mg.node_x[outlet_node_to_sample], mg.node_y[outlet_node_to_sample], "yo")
plt.plot(mg.node_x[upstream_node_to_sample], mg.node_y[upstream_node_to_sample], "bo")
plt.plot(
    mg.node_x[midstream_node_to_sample], mg.node_y[midstream_node_to_sample], "go"
)

In [ ]:

mg.at_node['surface_water__depth'].fill(1.e-12)  # a veneer of water stabilises the model
mg.at_node['surface_water__depth'] += mg.at_node['rainfall__flux'] * storm_t
    # storm_t here is the duration of the rainfall, from the rainfall component
    # We're going to assume the rainfall arrives effectively instantaneously, but
    # adding discharge during the run is completely viable

node_of_max_q = outlet_id #this is discharge point 
outlet_depth = []
outlet_times = []
post_storm_elapsed_time = x #(s)
last_storm_loop_tracker = 10

i = 0
while post_storm_elapsed_time < of_total_time:  #time in (s)
    
    dt = of.calc_time_step()
    of.run_one_step(dt=dt)
    post_storm_elapsed_time += dt
    hydrograph_time.append(post_storm_elapsed_time)
    print(post_storm_elapsed_time)
    storm_loop_tracker = post_storm_elapsed_time % 3600.  # show every 3 min
        # NB: Do NOT allow this plotting if there are multiple files in the folder
    if storm_loop_tracker < of_total_time:#last_storm_loop_tracker:
        #Instantiate figure as empty plot
        figelev = plt.figure()

        #Instantiate subplot as empty plot
        plot = plt.subplot()
        imshow_grid(mg,'surface_water__depth',var_name='Stage (m)')
        plt.title('Stage at t=' + str(post_storm_elapsed_time//1) + 's')
        plt.show()
        last_storm_loop_tracker = storm_loop_tracker
        outlet_depth.append(mg.at_node['surface_water__depth'][node_of_max_q])
        outlet_times.append(post_storm_elapsed_time)
        
        ## Append time and discharge to their lists to save data and for plotting.
        
        q = mg.at_link["surface_water__discharge"]
        discharge_at_outlet.append(np.abs(q[outlet_link_to_sample]) * mg.dx)
        discharge_upstream.append(np.abs(q[upstream_link_to_sample]) * mg.dx)
        discharge_midstream.append(np.abs(q[midstream_link_to_sample]) * mg.dx)
        print(post_storm_elapsed_time)


In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()
imshow_grid(mg,'surface_water__depth',var_name='Stage (m)')
plt.title('Stage at t=' + str(post_storm_elapsed_time//1) + 's')
plt.show()

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)

In [ ]:
## Code Block 7

## Calculate equilibrium discharge at each point for reference
outlet_eq_q = mg.at_node["drainage_area"][outlet_node_to_sample]
midstream_eq_q = (
    mg.at_node["drainage_area"][midstream_node_to_sample]
)
upstream_eq_q = (
    mg.at_node["drainage_area"][upstream_node_to_sample]
)


## Plotting hydrographs and equilibrium discharge
plt.figure(2)
plt.plot(hydrograph_time, discharge_at_outlet, "y-", label="outlet")
plt.plot(
    [np.min(hydrograph_time), np.max(hydrograph_time)],
    [outlet_eq_q, outlet_eq_q],
    "y--",
    label="outlet eq Q",
)
plt.plot(hydrograph_time, discharge_midstream, "g-", label="midstream")
plt.plot(
    [np.min(hydrograph_time), np.max(hydrograph_time)],
    [midstream_eq_q, midstream_eq_q],
    "g--",
    label="midstream eq Q",
)
plt.plot(hydrograph_time, discharge_upstream, "b-", label="upstream")
plt.plot([np.min(hydrograph_time), np.max(hydrograph_time)],
    [upstream_eq_q, upstream_eq_q],
    "b--",
    label="upstream eq Q",
)

## Plot storm end and center of storm for reference
plt.plot(
    [of_total_time, of_total_time], [0, 100], "k-", linewidth=2, label="storm end"
)
plt.plot(
    [of_total_time / 2, of_total_time / 2], [0, 100], "k:", label="storm mid point"
)

plt.ylabel("Discharge (cms)")
plt.xlabel("Time (seconds)")
plt.legend(loc="upper right")
title_text = "Hydrographs, Storm is " + ", Watershed is " 
plt.title(title_text)
plt.axis([0, np.max(hydrograph_time), 0, 100])